In this app we:
* Create a very simple [Planar Straight Line Graph](https://en.wikipedia.org/wiki/Planar_straight-line_graph) with the [ipyleaflet](https://github.com/jupygter-widgets/ipyleaflet) widget. In fact, it's just a polygon, and the rest of the app can only handle a single polygon at a time right now--so trash your existing polygon before creating another one. 
* Pull the terrain/bathymetry data for the area of interest from a global database. The notebook has code to use [GEBCO 2019](https://www.gebco.net). In fact, the app is pulling from a small grid around the initial coordinates to keep the docker container small.
* Generate a triangular mesh with the [triangle](https://www.cs.cmu.edu/~quake/triangle.html) mesh generator using the [triangle python bindings](https://rufat.be/triangle/index.html) and adaptively refine it to resolve the terrain data.
* Visualize the resulting mesh in several ways.
Find the code [here](https://github.com/pbugnion/voila-gallery/blob/master/mesh-generation-ipyleaflet/index.ipynb).

In [1]:
%matplotlib widget
import ipywidgets
from ipyleaflet import (
    Map,
    Polygon,
    DrawControl,
    WMSLayer
)
import numpy as np
import matplotlib.pyplot as plt
import triangle as tr
from matplotlib import tri as mpltr
from scipy import interpolate as scipy_interpolate
import ipyvolume as ipv
import ipympl

#initial map center and zoom level
center = [34.6252978589571, -77.34580993652344]
vertices=np.array([[-77.521791,  34.582935],
                   [-77.380628,  34.356501],
                   [-77.11704 ,  34.561348],
                   [-77.321914,  34.735944]])
zoom = 9
#use the GEBCO tile layers in the map widget--this is not necessary, just for viz
wms = WMSLayer(url='http://www.gebco.net/data_and_products/gebco_web_services/web_map_service/mapserv', 
               layers='GEBCO_LATEST', 
              )
#create a drawing control 
dc = DrawControl(polyline={},#
                 marker={},#{'shapeOptions': {'color': '#0000FF'}},
                 rectangle={},#{'shapeOptions': {'color': '#0000FF'}},
                 circle={},#{'shapeOptions': {'color': '#0000FF'}},
                 circlemarker={}#,
                 )

m = Map(center=center, zoom=zoom, layers=(wms,), controls=(dc,))

In [2]:
plt.ioff()
gridfig=plt.figure(num=0)
gridfig.set_label(' ')
#import h5py
#f = h5py.File('GEBCO_2019.nc','r')
f = np.load("terrain_slice_save.npz")
def plotGrid(button=None):
    global vertices,dc, j_min, j_max, i_min, i_max, lonr, latr, gridfig
    #get the vertices of the polygon
    if dc.last_draw['geometry'] != None:
        vertices=np.array(dc.last_draw['geometry']['coordinates'][0][:-1])
    lonr = [vertices[:,0].min(),vertices[:,0].max()]
    latr = [vertices[:,1].min(),vertices[:,1].max()]
    j_min = np.abs(f['lon'][:] - lonr[0]).argmin() - 1
    j_max = np.abs(f['lon'][:] - lonr[1]).argmin() + 1
    i_min = np.abs(f['lat'][:] - latr[0]).argmin() - 1
    i_max = np.abs(f['lat'][:] - latr[1]).argmin() + 1
    plt.figure(gridfig.number)
    plt.ion()
    plt.clf()
    plt.contourf(f['lon'][j_min:j_max],
                 f['lat'][i_min:i_max],
                 f['elevation'][i_min:i_max,
                                j_min:j_max])
    plt.colorbar()
    plt.title('')
plotGrid()
terrain_button  = ipywidgets.Button(
    description='Get Terrain',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Pull the terrain for the area of interest'
)
terrain_button.on_click(plotGrid)

In [3]:
plt.ioff()
meshfig=plt.figure(num=1,frameon=True)
meshfig.set_label(' ')
surffig=ipv.figure()
def genMesh(button=None):
    global vertices,meshfig
    nv = vertices.shape[0]
    vertexFlags=[1 for i in range(nv)]
    segments=[]
    segmentFlags=[]
    for i in range(nv):
        segments.append([i,(i+1)%nv])
        segmentFlags.append(1)
    nx = j_max - j_min
    ny = i_max - i_min
    bathy = np.zeros((nx*ny,3),'d')
    for i in range(ny):
        bathy[i*nx:(i+1)*nx,0] = f['lon'][j_min:j_max]
        bathy[i*nx:(i+1)*nx,1] = f['lat'][i_min+i]
    bathy[:,2] = np.reshape(f['elevation'][i_min:i_max,j_min:j_max], nx*ny)

    he = min((lonr[1]-lonr[0])/40.,(latr[1]-latr[0])/40.0)
    bathyGridDim = (ny,nx)
    x = bathy[:bathyGridDim[1],0]
    y = bathy[:bathyGridDim[0]*bathyGridDim[1]:bathyGridDim[1],1]
    z = bathy[:,2].reshape(bathyGridDim).transpose()
    bathyInterpolant = scipy_interpolate.RectBivariateSpline(x,y,z,kx=1,ky=1)
    pslg = dict(vertices=vertices,
                vertex_attributes=vertexFlags,
                segments=segments,
                segment_markers=segmentFlags)
    mesh = tr.triangulate(pslg,'pq30Da{0:f}'.format(0.5*he**2))
    area_max = 0.5*he**2
    mplmesh=mpltr.Triangulation(mesh['vertices'][:,0],
                                mesh['vertices'][:,1],
                                mesh['triangles'])
    not_converged=True
    tol=0.5
    while not_converged:
        mplmesh=mpltr.Triangulation(mesh['vertices'][:,0],
                                    mesh['vertices'][:,1],
                                    mesh['triangles'])
        trifinder = mpltr.TrapezoidMapTriFinder(mplmesh)
        zNodes = bathyInterpolant.ev(mesh['vertices'][:,0],
                                     mesh['vertices'][:,1])
        interpolator = mpltr.LinearTriInterpolator(mplmesh, zNodes, trifinder=trifinder)
        zInterp = interpolator(bathy[:,0],bathy[:,1])
        xE = bathy[np.abs(bathy[:,2] - zInterp) > tol,0]
        yE = bathy[np.abs(bathy[:,2] - zInterp) > tol,1]
        e = trifinder(xE,yE)
        area=np.ones((mesh['triangles'].shape[0],),'d')*0.5*he**2
        area_max *= 0.5
        area[e[e>=0]] = area_max
        mesh['triangle_max_area']=area
        mesh2 = tr.triangulate(mesh,'rpq30Da'.format(0.5*he**2))
        err = np.abs(bathy[:,2] - zInterp) > tol
        #print("L_infty_error",np.abs(bathy[:,2] - zInterp).max())
        not_converged = err.any()
        mesh=mesh2
    fig=plt.figure(meshfig.number)
    plt.ion()
    plt.clf()
    plt.triplot(mplmesh)
    fig = ipv.figure(surffig)
    if len(fig.meshes):
        del fig.meshes[-1]
    ipv.style.use('minimal')
    ipv.plot_trisurf(x=mesh['vertices'][:,0],
                          y=mesh['vertices'][:,1],
                          z=zNodes,
                          triangles=mesh['triangles'],
                          color='grey')
                   #,color=color, wireframe=wireframe)
    ipv.xlim(lonr[0], lonr[1])
    ipv.ylim(latr[0], latr[1])
    ipv.zlim(zNodes.min()*10,zNodes.max()*10)
    mesh3d = ipv.gcc()
button  = ipywidgets.Button(
    description='Generate Mesh',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Regenerate the mesh'
)
genMesh()
button.on_click(genMesh)

In [4]:
from ipywidgets import Tab,VBox
app=Tab(children=[m,
                  VBox(children=[terrain_button, gridfig.canvas]),
                  VBox(children=[button, meshfig.canvas]),
                  surffig])
app.set_title(0,"Map")
app.set_title(1,"Terrain Data")
app.set_title(2,"Mesh")
app.set_title(3,"Terrain Surface")
display(app)
